<a href="https://colab.research.google.com/github/uresha1995/Research-Methodology/blob/main/Assignment_2_22074260.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install the libraries

!pip install --upgrade transformers datasets

In [2]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
import re
import random

In [3]:
#Download the dataset

dataset = load_dataset("imdb")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [4]:
#Print  random reviews

sample_texts = [dataset["train"][i]["text"] for i in random.sample(range(25000), 5)]

for i, text in enumerate(sample_texts):
    print(f"--- Sample {i+1} ---\n{text[:500]}\n")

--- Sample 1 ---
This is the best Chinese movie I have ever seen, and, in my opinion, a lot better than Hero or Chrouching Tiger, Hidden Dragon. The movie is a unique combination of several genres: It's a beautiful love story, action movie, comedy and horror at the same time. And the most amazing thing is that it really succeeds in all of this!<br /><br />This movie definitely makes it to my top 5, and should be enjoyable to every movie lover. The action sequences do have the traditional unrealistic jumping and 

--- Sample 2 ---
The brilliant thing about Withnail & I is that it captures that not long left college and life could go either way moment along with all its other finery. Freebird is something for those who probably never considered higher education and just went straight to work aged 15/16. I know some of the broad sheets stuck the knife into this film when it came out in Cinemas but i saw it at a packed house in Hailsham and everybody seemed to really enjoy it. I grew up in

Preprocessing

In [5]:
#Remove empty and short reviews

def rem_empty(example):
    return len(example["text"].strip()) > 20

dataset["train"] = dataset["train"].filter(rem_empty)
dataset["test"] = dataset["test"].filter(rem_empty)

In [6]:
#Removing unnecessary spaces

def clean_text(example):
    example["text"] = example["text"].strip()
    return example

dataset = dataset.map(clean_text)

In [7]:
#Remove HTML break tags and replace with space

def clean_text(example):
    text = example["text"]
    text = re.sub(r"<br\s*/?>", " ", text)
    example["text"] = text
    return example

dataset = dataset.map(clean_text)

In [8]:
#Tokenization

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_fun(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = dataset.map(preprocess_fun, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Prepare data for Training

In [9]:
#Split dataset into training and test

train_data = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
test_data = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [10]:
#Loading BERT model

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
#!pip uninstall -y transformers

In [3]:
!pip install --quiet "transformers==4.41.0" "datasets==2.18.0" "evaluate==0.4.1"

In [1]:
from transformers import TrainingArguments

training_args = TrainingArguments(

    #Save the model outputs
    output_dir="./results",

    #Run evaluation of each epoch
    evaluation_strategy="epoch",

    #Log training progress every few steps
    logging_strategy="steps",

    #Logging to every 50 steps
    logging_steps=50,

    #Standard fine-tuning learning rate
    learning_rate=2e-5,

    #Tranining batch size
    per_device_train_batch_size=16,

    #Batch size for evaluation
    per_device_eval_batch_size=16,

    #Number of epochs
    num_train_epochs=2,

    # Weight decay for regularization
    weight_decay=0.01,

    #where to store logs
    logging_dir="./logs",

    #loading the best model
    load_best_model_at_end=True,

    #Save the epoch end of the model
    save_strategy="epoch",
    report_to="none"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [4]:
from transformers import Trainer, DataCollatorWithPadding

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
)

trainer.train()

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'EncoderDecoderCache' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)